# Tutorial: Translator Unit Model with Modular Property Package

In [ ]:
from IPython.core.display import SVG
SVG(filename='translator.svg')

## Learning Outcomes

- Demonstrate use of the Translator unit model in IDAES


## Problem Statement

In this tutorial, we will be transforming an inlet stream that uses the Benzene-Toluene Ideal property package to an outlet stream that uses a user-created configuration file. The state variables for each stream are as follows:

**Inlet State Variables(FTPx):**

Flow Rate = 100 mol/s

Temperature = 298 K

Pressure = 101325 Pa

Mole Fraction (benzene) = 0.6

Mole Fraction (toluene) = 0.4

**Outlet State Variables (FcTP):**

Benzene Molar Flow = 60 mol/s

Toluene Molar Flow = 40 mol/s

Temperature = 298 K

Pressure = 101325 Pa


For more details, please refer to the IDAES documentation: https://idaes-pse.readthedocs.io/en/stable

## Setting up the problem in IDAES

In the following cell, we will be importing the necessary components from Pyomo and IDAES.

In [ ]:
# Import objects from pyomo package 
from pyomo.environ import ConcreteModel, Constraint, value, SolverFactory

# Import the solver
from idaes.core.solvers import get_solver

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import the product unit model
from idaes.models.unit_models import Translator

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the modular property package to create a property block for the flowsheet
from idaes.models.properties.modular_properties.base.generic_property import GenericParameterBlock

# Import the BT_ideal property package to create a configuration file for the GenericParameterBlock based on the ideal eos
from idaes.models.properties.modular_properties.examples.BT_ideal import configuration as configuration_FTPx

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Create the ConcreteModel and the FlowsheetBlock objects, and attach the flowsheet block to it.
m = ConcreteModel()

m.fs = FlowsheetBlock(dynamic=False) # dynamic or ss flowsheet needs to be specified here


In the following cell, we will create a configuration file that is similar to the BT_ideal configuration, except this example will use FcTP state variables rather than FTPx state variables.

In [ ]:
# Import Component to create the benzene and toluene variables
from idaes.core import Component

# Import pyunits to define the units of variables
from pyomo.environ import units as pyunits

# Import potential phases
from idaes.core import LiquidPhase, VaporPhase

# Import the state definition
from idaes.models.properties.modular_properties.state_definitions import FcTP

# Import the ideal equation of state
from idaes.models.properties.modular_properties.eos.ideal import Ideal

# Import the phase equilibrium state
from idaes.models.properties.modular_properties.phase_equil import SmoothVLE

# Import the bubble dew method
from idaes.models.properties.modular_properties.phase_equil.bubble_dew import (
    IdealBubbleDew,
)

# Import the fugacity term
from idaes.models.properties.modular_properties.phase_equil.forms import fugacity

# Import the following literature references
import idaes.models.properties.modular_properties.pure.Perrys as Perrys
import idaes.models.properties.modular_properties.pure.RPP4 as RPP4
import idaes.models.properties.modular_properties.pure.NIST as NIST

configuration_FcTP = {
    "components": {
        "benzene": {
            "type": Component,
            "dens_mol_liq_comp": Perrys,
            "enth_mol_liq_comp": Perrys,
            "enth_mol_ig_comp": RPP4,
            "pressure_sat_comp": NIST,
            "phase_equilibrium_form": {("Vap", "Liq"): fugacity},
            "parameter_data": {
                "mw": 78.1136e-3,  # [1]
                "pressure_crit": 48.9e5,  # [1]
                "temperature_crit": 562.2,  # [1]
                "dens_mol_liq_comp_coeff": {
                    "eqn_type": 1,
                    "1": 1.0162,  # [2] pg. 2-98
                    "2": 0.2655,
                    "3": 562.16,
                    "4": 0.28212,
                },
                "cp_mol_ig_comp_coeff": {
                    "A": -3.392e1,  # [1]
                    "B": 4.739e-1,
                    "C": -3.017e-4,
                    "D": 7.130e-8,
                },
                "cp_mol_liq_comp_coeff": {
                    "1": 1.29e2,  # [2]
                    "2": -1.7e-1,
                    "3": 6.48e-4,
                    "4": 0,
                    "5": 0,
                },
                "enth_mol_form_liq_comp_ref": 49.0e3,  # [3]
                "enth_mol_form_vap_comp_ref": 82.9e3,  # [3]
                "pressure_sat_comp_coeff": {
                    "A": 4.72583,  # [NIST]
                    "B": 1660.652,
                    "C": -1.461,
                },
            },
        },
        "toluene": {
            "type": Component,
            "dens_mol_liq_comp": Perrys,
            "enth_mol_liq_comp": Perrys,
            "enth_mol_ig_comp": RPP4,
            "pressure_sat_comp": NIST,
            "phase_equilibrium_form": {("Vap", "Liq"): fugacity},
            "parameter_data": {
                "mw": 92.1405e-3,  # [1]
                "pressure_crit": 41e5,  # [1]
                "temperature_crit": 591.8,  # [1]
                "dens_mol_liq_comp_coeff": {
                    "eqn_type": 1,
                    "1": 0.8488,  # [2] pg. 2-98
                    "2": 0.26655,
                    "3": 591.8,
                    "4": 0.2878,
                },
                "cp_mol_ig_comp_coeff": {
                    "A": -2.435e1,
                    "B": 5.125e-1,
                    "C": -2.765e-4,
                    "D": 4.911e-8,
                },
                "cp_mol_liq_comp_coeff": {
                    "1": 1.40e2,  # [2]
                    "2": -1.52e-1,
                    "3": 6.95e-4,
                    "4": 0,
                    "5": 0,
                },
                "enth_mol_form_liq_comp_ref": 12.0e3,  # [3]
                "enth_mol_form_vap_comp_ref": 50.1e3,  # [3]
                "pressure_sat_comp_coeff": {
                    "A": 4.07827,  # [NIST]
                    "B": 1343.943,
                    "C": -53.773,
                },
            },
        },
    },
    "phases": {
        "Liq": {"type": LiquidPhase, "equation_of_state": Ideal},
        "Vap": {"type": VaporPhase, "equation_of_state": Ideal},
    },
    "base_units": {
        "time": pyunits.s,
        "length": pyunits.m,
        "mass": pyunits.kg,
        "amount": pyunits.mol,
        "temperature": pyunits.K,
    },
    "state_definition": FcTP,
    "state_bounds": {
        "flow_mol_comp": (0, 100, 1000, pyunits.mol / pyunits.s),
        "temperature": (273.15, 300, 450, pyunits.K),
        "pressure": (5e4, 1e5, 1e6, pyunits.Pa),
    },
    "pressure_ref": 1e5,
    "temperature_ref": 300,
    "phases_in_equilibrium": [("Vap", "Liq")],
    "phase_equilibrium_state": {("Vap", "Liq"): SmoothVLE},
    "bubble_dew_method": IdealBubbleDew,
}

In the following cell, we will be creating the translator unit model, assigning the appropriate property packages to it, and determining the degrees of freedom associated with the translator unit model.

In [ ]:
# Add properties parameter block to the flowsheet with the BT_ideal configuration file
m.fs.properties_FTPx = GenericParameterBlock(**configuration_FTPx)

# Add properties parameter block to the flowsheet with the configuration file created above
m.fs.properties_FcTP = GenericParameterBlock(**configuration_FcTP)

# Create an instance of the translator unit, attaching it to the flowsheet
# Specify that the property packages to be used for the inlet and outlet streams are the ones we created earlier

m.fs.translator = Translator(
    inlet_property_package=m.fs.properties_FTPx,
    outlet_property_package=m.fs.properties_FcTP,
)


# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)
print('The initial degrees of freedom are: {0}'.format(DOF_initial))

In [ ]:
assert DOF_initial == 9

In the following cell, we will be specifying the inlet conditions for the translator block.

In [ ]:
# Fix the inlet conditions

m.fs.translator.inlet.flow_mol.fix(100) # converting to mol/s as unit basis is mol/s
m.fs.translator.inlet.mole_frac_comp[0, "benzene"].fix(0.6)
m.fs.translator.inlet.mole_frac_comp[0, "toluene"].fix(0.4)
m.fs.translator.inlet.pressure.fix(101325) # Pa
m.fs.translator.inlet.temperature.fix(298) # K

In the following cell, we will be adding the translator block constraints to define the relationship between the inlet and outlet property packages and ensuring that there are zero degrees of freedom.

In [ ]:
# Define a translator block to make the constraints more readable
blk = m.fs.translator

# Add translator block constraints
blk.eq_benzene_balance = Constraint(
    expr=blk.properties_in[0].flow_mol * blk.properties_in[0].mole_frac_comp["benzene"]
    == blk.properties_out[0].flow_mol_comp["benzene"]
)
blk.eq_toluene_balance = Constraint(
    expr=blk.properties_in[0].flow_mol * blk.properties_in[0].mole_frac_comp["toluene"]
    == blk.properties_out[0].flow_mol_comp["toluene"]
)
blk.eq_equal_temperature = Constraint(
    expr=blk.properties_in[0].temperature == blk.properties_out[0].temperature
)
blk.eq_equal_pressure = Constraint(
    expr=blk.properties_in[0].pressure == blk.properties_out[0].pressure
)


# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print('The final degrees of freedom is: {0}'.format(DOF_final))

In [ ]:
assert DOF_final == 0

### Flowsheet Initialization

In [ ]:
# Initialize the flowsheet, and set the output at WARNING
m.fs.translator.initialize(outlvl=idaeslog.WARNING)

### Obtaining Simulation Results

In [ ]:
# Solve the simulation using the IDAES solver
# Note: If the degrees of freedom = 0, we have a square problem
solver = get_solver()
result = solver.solve(m, tee=True)

In [ ]:
from pyomo.opt import TerminationCondition, SolverStatus

# Check if termination condition is optimal
assert result.solver.termination_condition == TerminationCondition.optimal
assert result.solver.status == SolverStatus.ok

### View Results

As the following report will show, the inlet stream is represented with its five state variables (FTPx) while the outlet stream is represented with its four state variables (FcTP)

In [ ]:
# Display output report
m.fs.translator.report()


In [ ]:
import pytest

# Check inlet results
assert value(m.fs.translator.inlet.flow_mol[0]) == pytest.approx(100, rel=1e-6)
assert value(m.fs.translator.inlet.mole_frac_comp[0, "benzene"]) == pytest.approx(0.6, rel=1e-6)
assert value(m.fs.translator.inlet.mole_frac_comp[0, "toluene"]) == pytest.approx(0.4, rel=1e-6)

# Check outlet results
assert value(m.fs.translator.outlet.flow_mol_comp[0, "benzene"]) == pytest.approx(60, rel=1e-6)
assert value(m.fs.translator.outlet.flow_mol_comp[0, "toluene"]) == pytest.approx(40, rel=1e-6)
